# Phase Estimation Algorithm

The Quantum phase estimation algorithm (also referred to as quantum eigenvalue estimation algorithm), is a quantum algorithm to estimate the phase (or eigenvalue) of an eigenvector of a unitary operator.[1]

* Unitary operator: Matrix $U$ with $U^*U = UU^* = I$, with $I$ the identity matrix

It serves as a central building block for many quantum algorithms and implements a measurement to measure a more complex observable than only individual qubits.[2]

We've seen the phase estimation algorithm as a subroutine in Shor's algorithm.

## Table of Content
* Overview
    + Objective
    + Background: Why estimating eigenvalues?
    + General layout of the algoritm
* Implementation
* Examples
    + unitary 1-qubit gate (Cirq tutorial)
    + T-gate (Quiskit textbook)

## Overview
### Objective
For a given unitary operator $U$, the algorithm estimates $\theta$ in $U|\phi\rangle = e^{2\pi i\theta}|\phi\rangle$. Here $|\phi\rangle$ is an eigenvector and $2\pi i\theta$ is the corresponding eigenvalue. Since $U$ is unitary, all of its eigenvalues have a norm of $1$.[3]


### Bachground: Why estimating eigenvalues?
The goal is to map a complex mesurement onto a more simple setup for a measurement. For this the algorithm makes use of an observation in quantum mechanics: a momentum operator generates shifts in position for single particles. This observation can be used in quantum computing to shift a wave in such a way that it is shifted by the eigenvalue $\lambda$ of the applied unitary operator.[2]

=> Further reading keywords:
* John von Neumann’s measurement scheme
* [Unitarity (physics)](https://en.wikipedia.org/wiki/Unitarity_(physics))
* [(Unitary) time evolution](https://physics.stackexchange.com/questions/434883/what-is-meant-by-unitary-time-evolution)


### General layout of the algorithm
The algorithm prepares an eigenstate of the Hermitian operator in one register and stores the corresponding eigenvalue in a second register.[2]

=> we have two registers, each of them clustering multiple qubits

Steps:
* Setup / initalize the qubits
  * Create eigenvector state for all qubits on the first register
  * Create super position for all qubits on the second register
* Apply controlled unitary operations on the qubits on the second register
* Apply inverse quantum Fourier transformation on the first register
* Measure  the qubits on the first register




References:
+ [1] https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm
+ [2] https://quantum-computing.ibm.com/composer/docs/iqx/guide/quantum-phase-estimation
+ [3] https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html
+ [4] https://github.com/quantumlib/Cirq/blob/master/examples/phase_estimator.py
+ [5] https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html#example_t_gate

## Implementation

In [1]:
import cirq
import numpy as np

#### Class for the QPE Algorithm

In [2]:
class QuantumPhaseEstimator:
    
    def __init__(self, eigenstate, unitary_operator, amount_qubits):
        self.circuit = cirq.Circuit()
        self.ancilla = cirq.LineQubit(amount_qubits)
        self.count_qubits = cirq.LineQubit.range(amount_qubits)
        self.amount_qubits = amount_qubits
        
        self.init_qubits(eigenstate)
        self.apply_unitary_operator(unitary_operator)
        self.circuit.append(cirq.inverse(cirq.qft(*self.count_qubits, without_reverse=True)))
        self.circuit.append(cirq.measure(*self.count_qubits, key='phase'))

        
    def init_qubits(self, eigenstate):
        self.apply_superposition(self.count_qubits)
        self.circuit = eigenstate(self.circuit, [self.ancilla])

        
    def apply_superposition(self, qubits):
        self.circuit.append(cirq.H(q) for q in self.count_qubits)

        
    # algorithm quiskit textbook
    def apply_unitary_operator(self, operator):
        repetitions = 1
        for qubit in self.count_qubits:
            for j in range(repetitions):
                self.circuit.append(operator.on(self.ancilla).controlled_by(qubit))
            repetitions *= 2


    # algorithm from cirq example
    def apply_unitary_operator_v2(self, operator, amount_qubits):
        oracle_raised_to_power = [
            operator.on(self.ancilla).controlled_by(self.count_qubits[i]) ** (2 ** i) for i in range(len(self.count_qubits))
        ]
        circuit.append(oracle_raised_to_power)
    
    
    def get_circuit(self):
        return self.circuit
    
    
    def estimate(self, target, repetitions):
        result = {}
        
        simulator = cirq.Simulator()
        results = simulator.simulate(self.circuit)
    
        samples = simulator.run(self.circuit, repetitions=repetitions)
    
        # We now need to divide our result by 2^n to get θ:
        result['mode'] = samples.data['phase'].mode()[0]
        result['estimation'] = result['mode'] / 2 ** self.amount_qubits
        result['error'] = (target - result['estimation']) ** 2

        return result


#### Helper class to run experiments
This helper class runs an the QPE algorithm for a given setting. It also gives some printing outputs about to see how the algorithm performs.

In [3]:
class Experiment():
    
    def __init__(self):
        self.errors = []
    
    
    def run(self, target, number_qubits, eigenstate, operator, print_circuit=False):
        print(f'Testing with {number_qubits} qubits.')

        qpe = QuantumPhaseEstimator(eigenstate, operator, number_qubits)
        circuit = qpe.get_circuit()
        if(print_circuit):
            print(circuit)
        
        result = qpe.estimate(target, 1000)

        self.errors.append(result['error'])
        guess = result['estimation']
        mode = result['mode']
        print(f'target={target:0.4f}, estimate={guess:0.4f}={mode}/{2**number_qubits}')

        
    def print_error_rate(self):
        rms = np.sqrt(sum(self.errors) / len(self.errors))
        print(f'RMS Error: {rms:0.4f}\n')


## Example from Cirq tutorials [4]
An example unitary operator $U$:
* U: unitary 1-qubit gate 
* eigenvector |0>
* eigenvalue $e^{2\pi i\phi}$

$U|0\rangle = \begin{bmatrix} e^{2\pi i\phi} && 0 \\ 0 && 1 \end{bmatrix}  \begin{bmatrix} 1 \\ 0 \end{bmatrix} = e^{2\pi i\phi}|0\rangle$

The quantum phase estimator will estimate $\theta$ with:

$U|0\rangle = e^{2\pi i\theta}|0\rangle$

=> expected: $\theta = \phi$


### Implement the example operator and eigenstate

In [4]:
def example_gate(phi):
    gate = cirq.MatrixGate(matrix=np.array([[np.exp(2 * np.pi * 1.0j * phi), 0], [0, 1]]))
    return gate


def eigenvector_state(circuit, qubits):
    return circuit

### Visualize the circuit for two qubits

In [5]:
number_qubits = 2
qpe = QuantumPhaseEstimator(eigenvector_state, example_gate(0), number_qubits)
circuit = qpe.get_circuit()
print(circuit)

0: ───H───@─────────────────────────────────────────────────────qft[norev]^-1───M('phase')───
          │                                                     │               │
1: ───H───┼─────────────────@─────────────────@─────────────────#2──────────────M────────────
          │                 │                 │
          ┌             ┐   ┌             ┐   ┌             ┐
2: ───────│1.+0.j 0.+0.j│───│1.+0.j 0.+0.j│───│1.+0.j 0.+0.j│────────────────────────────────
          │0.+0.j 1.+0.j│   │0.+0.j 1.+0.j│   │0.+0.j 1.+0.j│
          └             ┘   └             ┘   └             ┘


### Run the circuit
Let's run a simulation equivalent to the cirq example:

* size register 1: 1
* size register 2: 8
* $\phi$ in range $[0, 1)$, step size: 0.1

In [6]:
number_qubits = 8
targets = np.arange(0, 1, 0.1)

experiment = Experiment()

for target in np.arange(0, 1, 0.1):
    experiment.run(target, number_qubits, eigenvector_state, example_gate(target))

experiment.print_error_rate()

Testing with 8 qubits.
target=0.0000, estimate=0.0000=0/256
Testing with 8 qubits.
target=0.1000, estimate=0.1016=26/256
Testing with 8 qubits.
target=0.2000, estimate=0.1992=51/256
Testing with 8 qubits.
target=0.3000, estimate=0.3008=77/256
Testing with 8 qubits.
target=0.4000, estimate=0.3984=102/256
Testing with 8 qubits.
target=0.5000, estimate=0.5000=128/256
Testing with 8 qubits.
target=0.6000, estimate=0.6016=154/256
Testing with 8 qubits.
target=0.7000, estimate=0.6992=179/256
Testing with 8 qubits.
target=0.8000, estimate=0.8008=205/256
Testing with 8 qubits.
target=0.9000, estimate=0.8984=230/256
RMS Error: 0.0011



## Example from Quiskit textbook [5]
* unitary operator: T-gate
* eigenvector: $|1\rangle$

The T-gate adds a phase of $e^{\frac{i\pi}{4}}$ to the state $|1\rangle$:

$T|1\rangle = \begin{bmatrix} 1 && 0 \\ 0 && e^{\frac{i\pi}{4}} \end{bmatrix}  \begin{bmatrix} 0 \\ 1 \end{bmatrix} = e^{\frac{i\pi}{4}}|1\rangle$

The quantum phase estimator will estimate $\theta$ with:

$T|1\rangle = e^2\pi i\theta|1\rangle$

=> expected: $\theta = \frac{1}{8}$

### Implement the eigenstate

In [7]:
def eigenvector_state(circuit, qubits):
    circuit.append(cirq.X(q) for q in qubits)
    return circuit

### Vizualise and run the circuit

In [8]:
number_qubits = 3
target = 1/8

experiment = Experiment()
experiment.run(target, number_qubits, eigenvector_state, cirq.ops.T, True)
experiment.print_error_rate()

Testing with 3 qubits.
0: ───H───@───────────────────────────qft[norev]^-1───M('phase')───
          │                           │               │
1: ───H───┼───@───@───────────────────#2──────────────M────────────
          │   │   │                   │               │
2: ───H───┼───┼───┼───@───@───@───@───#3──────────────M────────────
          │   │   │   │   │   │   │
3: ───X───T───T───T───T───T───T───T────────────────────────────────
target=0.1250, estimate=0.1250=1/8
RMS Error: 0.0000



## Summary
The quantum phase estimation algorithm is used to estimate eigenvalues of a given unitary operator $U$ and eigenstate $\phi$. Precisely it estimates $\theta$ in $U|\phi\rangle = e^{2\pi i\theta}|\phi\rangle$.

The eigenstate $\phi$ depends on the selected unitary operator $U$. This reflects on the initalization step of the algorithm.

4 Steps:
+ Initialize the qubits => apply correct eigenstates for ancilla qubits
+ Apply unitary operator
+ Apply inverse Fourier transformation
+ Measure counting qubits

The number of counting qubits reflects on the overall performance of the algorithm. To get a higher precision more counting qubits can be added. => On real devices this sets noise and performance limits we have today.

## Related Works
Tutorials with different implementations of the algorithm:
+ [4] (Cirq tutorial)
+ [3] (Quiskit textbook)
+ [2] (IBM quantum computing docs)
+ [Iterative QPE](https://qiskit.org/textbook/ch-labs/Lab04_IterativePhaseEstimation.html)

Web articles:
+ [Medium: Quantum Phase Estimation](https://medium.com/mit-6-s089-intro-to-quantum-computing/quantum-phase-estimation-7f6ab2beeae) (2020)

Doctor thesis:
+ [Quantum computing, phase estimation and applications](https://arxiv.org/pdf/0803.0909v1.pdf) (2008)


Examples: algorithms that uses QPE (variation) as subroutine:
+ Shor's algorithm (=> dicussed in this seminar)
+ [Quantum counting algorithm](https://en.wikipedia.org/wiki/Quantum_counting_algorithm)